http://jira.lge.com/issue/browse/HEDATAPLFM-1266

In [0]:
# %sql
# CREATE OR REPLACE FUNCTION aic_data_mart.private.convert_X_Device_Product_to_platform_version(X_Device_Product STRING, table_version STRING DEFAULT 'unknown')
# /*
# 작성자 : 김정욱 
# 목적 : 각 노말로그 테이블에서, platform_version을 추출하는 함수 
#  - bucket마다 데이터가 혼재되어 있으며, 이를 해결하기 위함
# PARAM 
#   X_Device_Product : string
#   table_version : string (ex. webOS21)
# */
# RETURNS STRING
# RETURN
#   CASE 
#     WHEN X_Device_Product = 'webOSTV 6.0' THEN 'webOS6.0'
#     WHEN X_Device_Product = 'webOSTV 7.0' THEN 'webOS22'
#     WHEN X_Device_Product = 'webOSTV 23' THEN 'webOS23'
#     WHEN X_Device_Product = 'webOSTV 24' THEN 'webOS24'
#     WHEN X_Device_Product = 'webOSTV 25' THEN 'webOS25'
#     WHEN X_Device_Product = 'webOSTV 26' THEN 'webOS26'
#     ELSE table_version 
#   END;

In [0]:
%sql
select * 
from aic_data_mart.tv_usage.iot_usage

In [0]:
%sql
select * 
from aic_data_fact.common_tv.tv_usage_monthly

In [0]:
%sql
-- IoT Usage Analysis for Nov 2025 - Jan 2026 (webOS22-25)
-- Using tv_usage_monthly for accurate total_activated_tvs count
-- Breakdown by Month, Country and Platform Version with Platform ID and Region

WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from aic_data_fact.common_tv.tv_usage_index
  )
  WHERE X_Device_Platform IS NOT NULL
),

platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
),

monthly_usage AS (
  -- Get monthly activated TVs from tv_usage_monthly
  SELECT 
    m.date_ym,
    m.country_code,
    m.platform_code,
    p.platform_version,
    SUM(m.one_day_plus_device) AS total_activated_tvs
  FROM aic_data_fact.common_tv.tv_usage_monthly m
  INNER JOIN platform_mapping p ON m.platform_code = p.platform_code
  WHERE m.date_ym IN ('2025-11', '2025-12', '2026-01')
  GROUP BY m.date_ym, m.country_code, m.platform_code, p.platform_version
),

iot_metrics AS (
  -- Get IoT device usage metrics
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    
    -- Users with IoT devices registered
    COUNT(DISTINCT CASE WHEN SIZE(iot_info) > 0 THEN mac_addr END) AS users_with_iot_devices,
    
    -- Users by each iot_spec
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'thinq'
    ) THEN mac_addr END) AS thinq_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'uei'
    ) THEN mac_addr END) AS uei_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'ghp'
    ) THEN mac_addr END) AS ghp_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'matter'
    ) THEN mac_addr END) AS matter_users
  
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version
)

-- Combine monthly usage with IoT metrics and add platform_id and region
SELECT 
  COALESCE(m.date_ym, i.date_ym) AS date_ym,
  COALESCE(m.country_code, i.country_code) AS country_code,
  cc.region,
  COALESCE(m.platform_version, i.platform_version) AS platform_version,
  COALESCE(m.platform_code, i.platform_code) AS platform_id,
  m.total_activated_tvs,
  i.users_with_iot_devices,
  i.thinq_users,
  i.uei_users,
  i.ghp_users,
  i.matter_users
FROM monthly_usage m
FULL OUTER JOIN iot_metrics i 
  ON m.date_ym = i.date_ym
  AND m.country_code = i.country_code 
  AND m.platform_code = i.platform_code
  AND m.platform_version = i.platform_version
LEFT JOIN aic_data_dimension.common.country_code cc
  ON COALESCE(m.country_code, i.country_code) = cc.country_code
ORDER BY date_ym, total_activated_tvs DESC;

In [0]:
%sql
-- IoT Usage Analysis for Nov 2025 - Jan 2026 (webOS22-25)
-- Using tv_usage_monthly for accurate total_activated_tvs count
-- Breakdown by Month, Country and Platform Version with Platform ID and Region

WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from aic_data_fact.common_tv.tv_usage_monthly
  )
  WHERE X_Device_Platform IS NOT NULL
),

platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  -- WHERE date_ym IN ('2025-12', '2026-01')
  --   AND platform_version IN ( 'webOS25')
),

monthly_usage AS (
  -- Get monthly activated TVs from tv_usage_monthly
  SELECT 
    m.date_ym,
    m.country_code,
    m.platform_code,
    p.platform_version,
    SUM(m.one_day_plus_device) AS total_activated_tvs
  FROM aic_data_fact.common_tv.tv_usage_monthly m
  INNER JOIN platform_mapping p ON m.platform_code = p.platform_code
  WHERE m.date_ym IN ('2025-11', '2025-12', '2026-01')
  -- WHERE m.date_ym IN ('2025-12', '2026-01')
  GROUP BY m.date_ym, m.country_code, m.platform_code, p.platform_version
),

iot_metrics AS (
  -- Get IoT device usage metrics
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    
    -- Users with IoT devices registered
    COUNT(DISTINCT CASE WHEN SIZE(iot_info) > 0 THEN mac_addr END) AS users_with_iot_devices,
    
    -- Users by each iot_spec
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'thinq'
    ) THEN mac_addr END) AS thinq_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'uei'
    ) THEN mac_addr END) AS uei_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'ghp'
    ) THEN mac_addr END) AS ghp_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'matter'
    ) THEN mac_addr END) AS matter_users
  
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  -- WHERE date_ym IN ('2025-12', '2026-01')
  --   AND platform_version IN ( 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version
)

-- Combine monthly usage with IoT metrics and add platform_id and region
SELECT 
  COALESCE(m.date_ym, i.date_ym) AS date_ym,
  COALESCE(m.country_code, i.country_code) AS country_code,
  cc.region,
  COALESCE(m.platform_version, i.platform_version) AS platform_version,
  COALESCE(m.platform_code, i.platform_code) AS platform_id,
  m.total_activated_tvs,
  i.users_with_iot_devices,
  i.thinq_users,
  i.uei_users,
  i.ghp_users,
  i.matter_users
FROM monthly_usage m
FULL OUTER JOIN iot_metrics i 
  ON m.date_ym = i.date_ym
  AND m.country_code = i.country_code 
  AND m.platform_code = i.platform_code
  AND m.platform_version = i.platform_version
LEFT JOIN aic_data_dimension.common.country_code cc
  ON COALESCE(m.country_code, i.country_code) = cc.country_code
ORDER BY date_ym, total_activated_tvs DESC;

In [0]:
%sql
-- IoT Usage Analysis for Nov 2025 - Jan 2026 (webOS22-25)
-- Breakdown by Month, Country and Platform Version with Platform ID and Region
-- Including device counts by iot_spec and iot_type

WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from aic_data_fact.common_tv.tv_usage_monthly
  )
  WHERE X_Device_Platform IS NOT NULL
),
platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
),

iot_device_counts AS (
  -- Count devices by iot_spec and iot_type
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    iot.iot_spec,
    iot.iot_type,
    COUNT(*) AS device_count
  FROM aic_data_mart.tv_usage.iot_usage
  LATERAL VIEW EXPLODE(iot_info) t AS iot
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version, iot.iot_spec, iot.iot_type
)

-- Device counts with region information
SELECT 
  d.date_ym,
  d.country_code,
  cc.region,
  d.platform_version,
  d.platform_code AS platform_id,
  d.iot_spec,
  d.iot_type,
  d.device_count
FROM iot_device_counts d
LEFT JOIN aic_data_dimension.common.country_code cc
  ON d.country_code = cc.country_code
ORDER BY date_ym, device_count DESC, iot_spec, iot_type;

In [0]:
%sql
select *
from aic_data_fact.common_tv.tv_usage_monthly
where 1=1
order by date_ym desc

In [0]:
%sql
select *, left(platform_code,3) as platform_version
from aic_data_fact.common_tv.tv_usage_monthly
where 1=1

In [0]:
%sql
select 
  left(platform_code,3) as platform_version
  , country_code
  , sum(one_day_plus_device)
  , sum(one_day_plus_user)
from aic_data_fact.common_tv.tv_usage_monthly
where 1=1
  and date_ym= '2026-01'
  and country_code = 'US'
group by 
  left(platform_code,3), country_code
order by
  1,2

In [0]:
%sql
select *
from aic_data_fact.common_tv.tv_usage_daily
where 1=1
order by date_ymd desc

In [0]:
%sql
create table if not exists sandbox.z_jeongjoo_lee.heds_1266_iot_user_cnt_dev as (

WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from aic_data_fact.common_tv.tv_usage_monthly
  )
  WHERE X_Device_Platform IS NOT NULL
),

platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  -- WHERE date_ym IN ('2025-12', '2026-01')
  --   AND platform_version IN ( 'webOS25')
),

monthly_usage AS (
  -- Get monthly activated TVs from tv_usage_monthly
  SELECT 
    m.date_ym,
    m.country_code,
    m.platform_code,
    p.platform_version,
    SUM(m.one_day_plus_device) AS total_activated_tvs
  FROM aic_data_fact.common_tv.tv_usage_monthly m
  INNER JOIN platform_mapping p ON m.platform_code = p.platform_code
  WHERE m.date_ym IN ('2025-11', '2025-12', '2026-01')
  -- WHERE m.date_ym IN ('2025-12', '2026-01')
  GROUP BY m.date_ym, m.country_code, m.platform_code, p.platform_version
),

iot_metrics AS (
  -- Get IoT device usage metrics
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    
    -- Users with IoT devices registered
    COUNT(DISTINCT CASE WHEN SIZE(iot_info) > 0 THEN mac_addr END) AS users_with_iot_devices,
    
    -- Users by each iot_spec
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'thinq'
    ) THEN mac_addr END) AS thinq_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'uei'
    ) THEN mac_addr END) AS uei_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'ghp'
    ) THEN mac_addr END) AS ghp_users,
    
    COUNT(DISTINCT CASE WHEN EXISTS(
      SELECT 1 FROM LATERAL VIEW EXPLODE(iot_info) t AS iot WHERE iot.iot_spec = 'matter'
    ) THEN mac_addr END) AS matter_users
  
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  -- WHERE date_ym IN ('2025-12', '2026-01')
  --   AND platform_version IN ( 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version
)

-- Combine monthly usage with IoT metrics and add platform_id and region
SELECT 
  COALESCE(m.date_ym, i.date_ym) AS date_ym,
  COALESCE(m.country_code, i.country_code) AS country_code,
  cc.region,
  COALESCE(m.platform_version, i.platform_version) AS platform_version,
  COALESCE(m.platform_code, i.platform_code) AS platform_id,
  m.total_activated_tvs,
  i.users_with_iot_devices,
  i.thinq_users,
  i.uei_users,
  i.ghp_users,
  i.matter_users
FROM monthly_usage m
FULL OUTER JOIN iot_metrics i 
  ON m.date_ym = i.date_ym
  AND m.country_code = i.country_code 
  AND m.platform_code = i.platform_code
  AND m.platform_version = i.platform_version
LEFT JOIN aic_data_dimension.common.country_code cc
  ON COALESCE(m.country_code, i.country_code) = cc.country_code
ORDER BY date_ym, total_activated_tvs DESC

)

In [0]:
%sql
create or replace table sandbox.z_jeongjoo_lee.heds_1266_iot_usage_dev as (
WITH platform_id_mapping AS (
  -- Get platform_id from normal_log tables
  SELECT DISTINCT 
    X_Device_Platform AS platform_id,
    X_Device_Platform AS platform_code
  FROM (
    select distinct platform_code as X_Device_Platform
    from aic_data_fact.common_tv.tv_usage_monthly
  )
  WHERE X_Device_Platform IS NOT NULL
),
platform_mapping AS (
  -- Get platform_code to platform_version mapping
  SELECT DISTINCT 
    platform_code,
    platform_version
  FROM aic_data_mart.tv_usage.iot_usage
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
),

iot_device_counts AS (
  -- Count devices by iot_spec and iot_type
  SELECT 
    date_ym,
    country_code,
    platform_code,
    platform_version,
    iot.iot_spec,
    iot.iot_type,
    COUNT(*) AS device_count
  FROM aic_data_mart.tv_usage.iot_usage
  LATERAL VIEW EXPLODE(iot_info) t AS iot
  WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
    AND platform_version IN ('webOS22', 'webOS23', 'webOS24', 'webOS25')
  GROUP BY date_ym, country_code, platform_code, platform_version, iot.iot_spec, iot.iot_type
)

-- Device counts with region information
SELECT 
  d.date_ym,
  d.country_code,
  cc.region,
  d.platform_version,
  d.platform_code AS platform_id,
  d.iot_spec,
  d.iot_type,
  d.device_count
FROM iot_device_counts d
LEFT JOIN aic_data_dimension.common.country_code cc
  ON d.country_code = cc.country_code
ORDER BY date_ym, device_count DESC, iot_spec, iot_type

)

In [0]:
%sql
create table if not exists sandbox.z_jeongjoo_lee.heds_1266_tv_usage_monthly as (
  select *
  from aic_data_fact.common_tv.tv_usage_monthly
)

In [0]:
%sql
WITH exploded AS (
  SELECT
    date_ym,
    country_code,
    platform_code,
    platform_version,
    mac_addr,
    iot.iot_spec
  FROM aic_data_mart.tv_usage.iot_usage
  LATERAL VIEW OUTER EXPLODE(iot_info) t AS iot
  -- WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
  --   AND platform_version IN ('webOS22','webOS23','webOS24','webOS25')
  WHERE date_ym IN ('2026-01')
    AND platform_version IN ('webOS25')
),

iot_metrics AS (
  SELECT
    date_ym,
    country_code,
    platform_code,
    platform_version,

    COUNT(DISTINCT mac_addr) AS users_with_iot_devices,

    COUNT(DISTINCT CASE WHEN iot_spec = 'thinq'  THEN mac_addr END) AS thinq_users,
    COUNT(DISTINCT CASE WHEN iot_spec = 'uei'    THEN mac_addr END) AS uei_users,
    COUNT(DISTINCT CASE WHEN iot_spec = 'ghp'    THEN mac_addr END) AS ghp_users,
    COUNT(DISTINCT CASE WHEN iot_spec = 'matter' THEN mac_addr END) AS matter_users

  FROM exploded
  GROUP BY date_ym, country_code, platform_code, platform_version
)

SELECT *
FROM iot_metrics
ORDER BY date_ym, users_with_iot_devices DESC;

In [0]:
%sql
-- IoT Usage Ratio Analysis for Nov 2025 - Jan 2026 (webOS22-25)
-- Denominator: tv_usage_monthly (total activated TVs)
-- Numerator: iot_usage (users with IoT devices, spec breakdown)
-- Anchor: monthly_usage, Join: LEFT JOIN, Missing IoT -> 0

WITH params AS (
  SELECT
    array('2025-11','2025-12','2026-01') AS months,
    array('webOS22','webOS23','webOS24','webOS25') AS versions
),

platform_mapping AS (
  -- Ensure 1:1 mapping per platform_code to avoid row explosion
  -- If platform_code can map to multiple versions, pick a deterministic one.
  SELECT
    platform_code,
    MAX(platform_version) AS platform_version
  FROM aic_data_mart.tv_usage.iot_usage
  -- WHERE date_ym IN ('2025-11', '2025-12', '2026-01')
  --   AND platform_version IN ('webOS22','webOS23','webOS24','webOS25')
  WHERE date_ym IN ('2025-12','2026-01')
    AND platform_version IN ('webOS25')
  GROUP BY platform_code
),

monthly_usage AS (
  -- Denominator: monthly activated TVs
  SELECT
    m.date_ym,
    m.country_code,
    m.platform_code,
    pm.platform_version,
    SUM(m.one_day_plus_device) AS total_activated_tvs
  FROM aic_data_fact.common_tv.tv_usage_monthly m
  INNER JOIN platform_mapping pm
    ON m.platform_code = pm.platform_code
  -- WHERE m.date_ym IN ('2025-11','2025-12','2026-01')
  WHERE m.date_ym IN ('2025-12','2026-01')

  GROUP BY m.date_ym, m.country_code, m.platform_code, pm.platform_version
),

-- Build per-user spec presence (avoid double-counting when multiple iot_info entries exist)
user_spec_presence AS (
  SELECT
    u.date_ym,
    u.country_code,
    u.platform_code,
    u.platform_version,
    u.mac_addr,
    collect_set(iot.iot_spec) AS specs
  FROM aic_data_mart.tv_usage.iot_usage u
  LATERAL VIEW OUTER EXPLODE(u.iot_info) t AS iot
  WHERE u.date_ym IN ('2025-11','2025-12','2026-01')
    AND u.platform_version IN ('webOS22','webOS23','webOS24','webOS25')
  GROUP BY u.date_ym, u.country_code, u.platform_code, u.platform_version, u.mac_addr
),

iot_metrics AS (
  -- Numerator: users with any IoT devices + spec breakdown
  SELECT
    date_ym,
    country_code,
    platform_code,
    platform_version,

    -- users with any iot devices registered
    COUNT(DISTINCT CASE WHEN size(specs) > 0 THEN mac_addr END) AS users_with_iot_devices,

    -- spec users (presence-based, not per-device)
    COUNT(DISTINCT CASE WHEN array_contains(specs, 'thinq')  THEN mac_addr END) AS thinq_users,
    COUNT(DISTINCT CASE WHEN array_contains(specs, 'uei')    THEN mac_addr END) AS uei_users,
    COUNT(DISTINCT CASE WHEN array_contains(specs, 'ghp')    THEN mac_addr END) AS ghp_users,
    COUNT(DISTINCT CASE WHEN array_contains(specs, 'matter') THEN mac_addr END) AS matter_users

  FROM user_spec_presence
  GROUP BY date_ym, country_code, platform_code, platform_version
)

SELECT
  m.date_ym,
  m.country_code,
  cc.region,
  m.platform_version,
  m.platform_code AS platform_id,

  m.total_activated_tvs,

  COALESCE(i.users_with_iot_devices, 0) AS users_with_iot_devices,
  COALESCE(i.thinq_users, 0)  AS thinq_users,
  COALESCE(i.uei_users, 0)    AS uei_users,
  COALESCE(i.ghp_users, 0)    AS ghp_users,
  COALESCE(i.matter_users, 0) AS matter_users,

  -- Ratios
  ROUND(COALESCE(i.users_with_iot_devices, 0) / m.total_activated_tvs, 6) AS iot_usage_ratio,
  ROUND(COALESCE(i.thinq_users, 0)  / m.total_activated_tvs, 6) AS thinq_ratio,
  ROUND(COALESCE(i.uei_users, 0)    / m.total_activated_tvs, 6) AS uei_ratio,
  ROUND(COALESCE(i.ghp_users, 0)    / m.total_activated_tvs, 6) AS ghp_ratio,
  ROUND(COALESCE(i.matter_users, 0) / m.total_activated_tvs, 6) AS matter_ratio

FROM monthly_usage m
LEFT JOIN iot_metrics i
  ON m.date_ym = i.date_ym
 AND m.country_code = i.country_code
 AND m.platform_code = i.platform_code
 AND m.platform_version = i.platform_version
LEFT JOIN aic_data_dimension.common.country_code cc
  ON m.country_code = cc.country_code
ORDER BY m.date_ym, m.total_activated_tvs DESC;